<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Custom embedddings con Gensim



### Objetivo
El objetivo es utilizar documentos / corpus para crear embeddings de palabras basado en ese contexto. Se utilizará canciones de bandas para generar los embeddings, es decir, que los vectores tendrán la forma en función de como esa banda haya utilizado las palabras en sus canciones.

In [2]:
!pip install gensim==3.8.3

     |████████████████████████████████| 24.2 MB 10.0 MB/s 
  Using cached smart_open-5.2.1-py3-none-any.whl (58 kB)
You should consider upgrading via the '/Users/ext_asalassa/.pyenv/versions/3.7.11/bin/python3.7 -m pip install --upgrade pip' command.


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec

### Datos
Utilizaremos como dataset canciones de bandas de habla inglés.

In [2]:
# Descargar la carpeta de dataset
import os
import platform
if os.access('./songs_dataset', os.F_OK) is False:
    if os.access('songs_dataset.zip', os.F_OK) is False:
        if platform.system() == 'Windows':
            !curl https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip -o songs_dataset.zip
        else:
            !wget songs_dataset.zip https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/datasets/songs_dataset.zip
    !unzip -q songs_dataset.zip   
else:
    print("El dataset ya se encuentra descargado")

El dataset ya se encuentra descargado


In [3]:
# Posibles bandas
list_dir = os.listdir("./songs_dataset/")
print(len(list_dir))
list_dir

49


['prince.txt',
 'dickinson.txt',
 'notorious-big.txt',
 'beatles.txt',
 'bob-dylan.txt',
 'bjork.txt',
 'johnny-cash.txt',
 'disney.txt',
 'janisjoplin.txt',
 'kanye.txt',
 'bob-marley.txt',
 'leonard-cohen.txt',
 'ludacris.txt',
 'adele.txt',
 'alicia-keys.txt',
 'joni-mitchell.txt',
 'amy-winehouse.txt',
 'lorde.txt',
 'rihanna.txt',
 'Kanye_West.txt',
 'nirvana.txt',
 'cake.txt',
 'bieber.txt',
 'notorious_big.txt',
 'missy-elliott.txt',
 'dolly-parton.txt',
 'jimi-hendrix.txt',
 'michael-jackson.txt',
 'al-green.txt',
 'lil-wayne.txt',
 'lady-gaga.txt',
 'lin-manuel-miranda.txt',
 'nursery_rhymes.txt',
 'dj-khaled.txt',
 'radiohead.txt',
 'patti-smith.txt',
 'blink-182.txt',
 'Lil_Wayne.txt',
 'dr-seuss.txt',
 'r-kelly.txt',
 'drake.txt',
 'britney-spears.txt',
 'bruce-springsteen.txt',
 'nicki-minaj.txt',
 'kanye-west.txt',
 'paul-simon.txt',
 'nickelback.txt',
 'eminem.txt',
 'bruno-mars.txt']

In [4]:
# Armar el dataset utilizando salto de línea para separar las oraciones/docs
df = pd.read_csv('songs_dataset/michael-jackson.txt', sep='/n', header=None)
df.head()

/Users/ext_asalassa/.pyenv/versions/3.7.11/lib/python3.7/site-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


,0
0,[Spoken Intro:]
1,You ever want something
2,that you know you shouldn't have
3,"The more you know you shouldn't have it,"
4,The more you want it


In [5]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 9484


### 1 - Preprocesamiento

In [6]:
from keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

In [7]:
# Demos un vistazo
sentence_tokens[:2]

[['spoken', 'intro'], ['you', 'ever', 'want', 'something']]

### 2 - Crear los vectores (word2vec)

In [8]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobracargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [9]:
# Crearmos el modelo generador de vectoeres
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,       # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,          # cant de palabras antes y desp de la predicha
                     size=300,          # dimensionalidad de los vectores 
                     negative=20,       # cantidad de negative samples... 0 es no se usa
                     workers=1,         # si tienen más cores pueden cambiar este valor
                     sg=1)              # modelo 0:CBOW  1:skipgram

In [10]:
# Buildear el vocabularui con los tokens
w2v_model.build_vocab(sentence_tokens)

In [11]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 9484


In [12]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.vocab))

Cantidad de words distintas en el corpus: 1000


### 3 - Entrenar el modelo generador

In [13]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=20,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 337560.65625
Loss after epoch 1: 227757.09375
Loss after epoch 2: 228931.0
Loss after epoch 3: 225456.8125
Loss after epoch 4: 196352.8125
Loss after epoch 5: 188361.25
Loss after epoch 6: 181669.5
Loss after epoch 7: 174853.25
Loss after epoch 8: 171838.25
Loss after epoch 9: 165783.875
Loss after epoch 10: 147294.0
Loss after epoch 11: 145497.25
Loss after epoch 12: 143081.0
Loss after epoch 13: 141314.25
Loss after epoch 14: 140920.0
Loss after epoch 15: 138942.25
Loss after epoch 16: 139251.75
Loss after epoch 17: 137726.25
Loss after epoch 18: 138762.0
Loss after epoch 19: 137523.75


(626341, 993780)

### 4 - Ensayar

In [14]:
# Palabras que MÁS se relacionan con...:
print("Palabras que MÁS se relacionan con...:")
print("* thriller \n", w2v_model.wv.most_similar(positive=["thriller"], topn=10))
print("* remember \n", w2v_model.wv.most_similar(positive=["remember"], topn=10))
print("* bad \n", w2v_model.wv.most_similar(positive=["bad"], topn=10))

Palabras que MÁS se relacionan con...:
* thriller 
 [('winds', 0.8471239805221558), ('till', 0.8416576385498047), ('killer', 0.8151499032974243), ('darkness', 0.8041678667068481), ('big', 0.7986006736755371), ('breaking', 0.7833280563354492), ('history', 0.7809442281723022), ('injustice', 0.7768913507461548), ('lovers', 0.7725609540939331), ('s', 0.7716856002807617)]
* remember 
 [('doot', 0.7745230793952942), ('after', 0.7475050687789917), ('means', 0.7279583215713501), ('times', 0.7249292731285095), ('great', 0.7235779762268066), ('thrill', 0.717964231967926), ('perfect', 0.7160735726356506), ('choice', 0.6902207136154175), ('play', 0.6886769533157349), ('end', 0.6703741550445557)]
* bad 
 [('shameone', 0.9027756452560425), ('smooth', 0.8853279948234558), ('laughing', 0.8116799592971802), ('fast', 0.8079899549484253), ('bothered', 0.7920960783958435), ('telling', 0.7811301946640015), ('easy', 0.7695871591567993), ('everyday', 0.762784481048584), ('late', 0.7604116201400757), ('true',

In [15]:
# Palabras que MENOS se relacionan con...:
print("Palabras que MENOS se relacionan con...:")
print("* thriller \n", w2v_model.wv.most_similar(negative=["thriller"], topn=10))
print("* remember \n", w2v_model.wv.most_similar(negative=["remember"], topn=10))
print("* bad \n", w2v_model.wv.most_similar(negative=["bad"], topn=10))

Palabras que MENOS se relacionan con...:
* thriller 
 [('cheater', 0.07655061036348343), ('refrain', 0.019761323928833008), ('pitbull', 0.007705641910433769), ('uuh', 0.005732991732656956), ('stevie', -0.010979164391756058), ('siedah', -0.026299960911273956), ('happens', -0.09750251471996307), ('let', -0.10877986997365952), ('whatever', -0.11904726177453995), ("g'on", -0.1207893118262291)]
* remember 
 [('cheater', 0.04230940341949463), ('pitbull', 0.03555934131145477), ('stevie', 0.02836664766073227), ('refrain', -0.024253804236650467), ('siedah', -0.06023828685283661), ('uuh', -0.0710870623588562), ('high', -0.10746919363737106), ('your', -0.11496143788099289), ('his', -0.1173308715224266), ('enough', -0.1291123777627945)]
* bad 
 [('cheater', 0.0656174048781395), ('pitbull', 0.03338500112295151), ('siedah', 0.025273408740758896), ('refrain', 0.004684077575802803), ('stevie', -0.0158399548381567), ('uuh', -0.0575295053422451), ('blow', -0.08758994936943054), ('her', -0.09087440371513

In [16]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["billie"], topn=10)

[('jean', 0.9890145063400269),
 ('son', 0.9004597663879395),
 ('lover', 0.871423602104187),
 ('kid', 0.8672822713851929),
 ('sleep', 0.8449476957321167),
 ('yours', 0.8257718086242676),
 ('dangerous', 0.822664737701416),
 ('dom', 0.8086676597595215),
 ('sheldon', 0.806425929069519),
 ('s', 0.8063123226165771)]

In [17]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["dance"], topn=5)

[('floor', 0.83399897813797),
 ('across', 0.8255425691604614),
 ('earth', 0.7911186814308167),
 ('carpet', 0.7882789373397827),
 ('greatest', 0.7828449010848999)]

In [18]:
# Ensayar con una palabra que no está en el corpus (en vocab):
w2v_model.wv.most_similar(negative=["diedaa"])

KeyError: "word 'diedaa' not in vocabulary"

### 5 - Visualizar agrupación de vectores

In [19]:
from sklearn.decomposition import IncrementalPCA    
from sklearn.manifold import TSNE                   
import numpy as np                                  

def reduce_dimensions(model):
    num_dimensions = 2  

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index2word)  

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
    return x_vals, y_vals, labels

In [20]:
!pip install plotly nbformat==4.2.0

You should consider upgrading via the '/Users/ext_asalassa/.pyenv/versions/3.7.11/bin/python3.7 -m pip install --upgrade pip' command.


In [21]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

x_vals, y_vals, labels = reduce_dimensions(w2v_model)

/Users/ext_asalassa/.pyenv/versions/3.7.11/lib/python3.7/site-packages/sklearn/manifold/_t_sne.py:783: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  FutureWarning,
/Users/ext_asalassa/.pyenv/versions/3.7.11/lib/python3.7/site-packages/sklearn/manifold/_t_sne.py:793: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  FutureWarning,


In [24]:
MAX_WORDS=200
fig = px.scatter(x=x_vals[:MAX_WORDS], y=y_vals[:MAX_WORDS], text=labels[:MAX_WORDS])
fig.show() # esto para plotly en colab